In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.inspection import permutation_importance
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import ttest_ind
from sklearn.preprocessing import MinMaxScaler


In [ ]:
user_actions = pd.read_csv('/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/data/user_actions.csv')
homes = pd.read_csv('/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/data/homes_new.csv')

In [3]:
# Нормализация числовых признаков
numeric_features = ['price', 'level', 'rooms', 'area', 'kitchen_area', 'geo_sum']
scaler = StandardScaler()
homes[numeric_features] = scaler.fit_transform(homes[numeric_features])

In [ ]:
homes = homes[['price',	'level', 'rooms', 'area',	'kitchen_area',	'geo_sum', 'id']]

In [5]:
homes

,price,level,rooms,area,kitchen_area,geo_sum,id
0,-0.384262,1.776204,-2.065946,-0.986463,-0.014410,-9.848709,1
1,-0.381294,1.497514,-2.065946,-0.981173,-0.014410,-9.848709,2
2,-0.389474,0.661443,-2.065946,-0.985260,-0.014410,-9.848709,3
3,-0.404111,1.776204,-2.065946,-0.986463,-0.014410,-9.848709,4
4,-0.376288,1.218824,-2.065946,-0.942219,-0.014410,-9.848709,5
...,...,...,...,...,...,...,...
9995,2.594037,-0.871353,3.908234,6.345961,1.274461,0.284550,9996
9996,1.385781,1.358169,3.908234,4.109733,0.731611,0.492606,9997
9997,2.284744,-0.174627,4.655007,7.884871,0.988751,-0.224879,9998
9998,0.152379,-0.453318,4.655007,4.085688,0.493520,0.086122,9999


In [ ]:
import pandas as pd
import sys
sys.path.append('.')  # Добавляем текущую директорию в путь поиска модулей
from utils import search_history
from utils import make_recommendations


user_actions = search_history(user_id=666, user_type=3)
homes = pd.read_csv('data/homes_fixed_scaled.csv')
recs = make_recommendations(homes, user_actions)

utils.py загружен


/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



In [26]:
# Объединение данных по объектам
homes_fixed.set_index('id', inplace=True)

# Группировка действий пользователей
user_profiles = {}
for user, actions in user_actions.groupby('user_id'):
    interacted_obj_ids = actions['obj_id'].unique()
    interacted_objects = homes_fixed.loc[homes_fixed.index.intersection(interacted_obj_ids)]

    if not interacted_objects.empty:
        user_profiles[user] = interacted_objects.mean(axis=0)

# Преобразование профилей пользователей в DataFrame
user_profiles_df = pd.DataFrame.from_dict(user_profiles, orient='index')

# Вычисление косинусного сходства между пользователем и объектами
similarity_matrix = cosine_similarity(user_profiles_df, homes_fixed)

# Создание DataFrame с рекомендациями
recommendations = pd.DataFrame(similarity_matrix, index=user_profiles_df.index, columns=homes_fixed.index)

# Выбор 10 объектов, с которыми пользователь не взаимодействовал
final_recommendations = {}
for user in user_profiles_df.index:
    interacted = set(user_actions[user_actions['user_id'] == user]['obj_id'])
    sorted_recs = recommendations.loc[user].drop(index=interacted).nlargest(10)
    final_recommendations[user] = sorted_recs.index.tolist()

# Вывод рекомендаций (id)
final_recommendations


/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



{999: [410, 54, 183, 564, 595, 60, 62, 66, 86, 234]}

Ниже оценка рек. системы с помощью стат. теста 


In [ ]:
# Создадим структуру для хранения результатов тестов
stat_test_results = {}

# Выполнение t-теста для каждого пользователя
for user in final_recommendations.keys():
    interacted_obj_ids = user_actions[user_actions['user_id'] == user]['obj_id']
    recommended_obj_ids = final_recommendations[user]

    interacted_objects = homes_fixed.loc[homes_fixed.index.intersection(interacted_obj_ids)]
    recommended_objects = homes_fixed.loc[homes_fixed.index.intersection(recommended_obj_ids)]

    if not interacted_objects.empty and not recommended_objects.empty:
        # Средние значения по объектам
        interacted_mean = interacted_objects.mean(axis=0)
        recommended_mean = recommended_objects.mean(axis=0)

        # Проведение t-теста
        t_stat, p_val = ttest_ind(interacted_mean, recommended_mean, equal_var=False, nan_policy='omit')

        # Сохранение результатов
        stat_test_results[user] = {'t_stat': t_stat, 'p_val': p_val}

        # Вывод результатов для каждого пользователя
        print(f'User {user}: t_stat = {t_stat:.5f}, p_val = {p_val:.5f}')

# Преобразуем в DataFrame для удобного анализа
stat_test_results_df = pd.DataFrame.from_dict(stat_test_results, orient='index')

# Подсчет доли пользователей, у которых p_val > 0.05
no_significant_diff_ratio = (stat_test_results_df['p_val'] > 0.05).mean()
print(f'Доля пользователей, у которых нет статистически значимой разницы: {no_significant_diff_ratio:.2%}')


User 1: t_stat = 0.24174, p_val = 0.81505
User 2: t_stat = -0.14729, p_val = 0.88584
User 3: t_stat = -1.20257, p_val = 0.26678
User 4: t_stat = -0.65137, p_val = 0.53312
User 5: t_stat = -0.89812, p_val = 0.40147
Доля пользователей, у которых нет статистически значимой разницы: 100.00%


если p_val > 0.05, то нет оснований полагать, что есть стат. значимая разница => рекомендации были хорошими для всех юзеров